In [ ]:
from fastcore.utils import *
import fastcore.all as fc, re, math, itertools, functools, numpy as np, types, typing, dataclasses, matplotlib.pyplot as plt, collections, regex
from regex import search
from collections import Counter
from collections.abc import Iterable
np.set_printoptions(linewidth=150, suppress=True)
plt.rcParams['figure.dpi'] = 50

In [ ]:
!ls static

Understanding-ISO-17025-for-Laser-Power-and-Energy-Measurement-043520ed-a855-43f6-b05c-a9c83afded79.pdf
enhanced_laser_nodes.pkl
labmax-touch-ds.pdf
laser_nodes.db
laser_qdrant.pkl
multipurpose-energymax-sensors-usb-rs232-ds.pdf
parsed_lmc_docs.pkl
pm10k-plus-ds.pdf
requirements.txt


In [ ]:
from fastcore.utils import *
path = Path('static')

In [ ]:
post_fnames = path.ls(file_exts=['.pdf'])
post_fnames

(#4) [Path('static/labmax-touch-ds.pdf'),Path('static/multipurpose-energymax-sensors-usb-rs232-ds.pdf'),Path('static/pm10k-plus-ds.pdf'),Path('static/Understanding-ISO-17025-for-Laser-Power-and-Energy-Measurement-043520ed-a855-43f6-b05c-a9c83afded79.pdf')]

In [ ]:
!pip install llama-cloud-services llama-index-core llama-index-readers-file

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
datasheet_parse_prompt_improved='''# CRITICAL PARSING INSTRUCTIONS - FOLLOW EXACTLY

These documents contain technical information about laser power meters, laser energy meters, and laser beam diagnostics products.

## TABLE FORMATTING RULES (HIGHEST PRIORITY):

1. FILL ALL EMPTY CELLS: Every cell in specification tables must be filled. No cell should be empty.
   - When a value spans multiple columns, copy that value to each individual cell it applies to.
   - Example: If "0.19 to 12" appears once but applies to all models, it must be repeated in each model's column.

2. TABLE STRUCTURE: Include model names in the first row of each column above specifications.
   - Example: |Model|PM2|PM10|PM30|

3. PART NUMBERS: 
   - Keep part numbers within specification tables
   - Remove any footnote symbols/superscripts from part numbers
   - Most part numbers have seven digits unless they start with 33 and include dashes

## EXAMPLES OF CORRECT TABLE FORMATTING:

INCORRECT (with empty cells):
|Wavelength Range (µm)| |0.19 to 12| | |
|Active Area Diameter (mm)|50| |25|10|

CORRECT (all cells filled):
|Wavelength Range (µm)|0.19 to 12|0.19 to 12|0.19 to 12|0.19 to 12|
|Active Area Diameter (mm)|50|50|25|10|

## ADDITIONAL INSTRUCTIONS:

1. Rename "Mechanical Specifications" to "Mechanical Dimensions"
2. Leave drawing dimensions as text, not in table format
3. Ignore footnotes with underscore numbers in part numbers

## VALIDATION STEP:
After completing the table, verify that:
1. Every cell contains a value (no empty cells)
2. Values that span multiple columns are copied to each relevant cell
3. All part numbers are properly formatted without superscripts

Please confirm you've followed these instructions with the statement: "All cells filled and formatted according to instructions."
'''

In [ ]:
import os
from llama_cloud_services import LlamaParse

gpt4o_parser = LlamaParse(
    result_type="markdown",
    verbose=True,
    invalidate_cache=True,
    do_not_cache=True,
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="openai-gpt4o",
    vendor_multimodal_api_key=os.environ.get("OPENAI_API_KEY"),
    user_prompt=datasheet_parse_prompt_improved,
)

In [ ]:
import asyncio, threading
from llama_cloud_services import LlamaParse
parser = LlamaParse(result_type="markdown")
def run_in_thread(coro):
    res = None
    def target():
        nonlocal res
        asyncio.set_event_loop_policy(asyncio.DefaultEventLoopPolicy())
        loop = asyncio.new_event_loop()
        res = loop.run_until_complete(coro)
        loop.close()
    t = threading.Thread(target=target)
    t.start(); t.join()
    return res

In [ ]:
for i, doc in enumerate(doc, 1):
    print(f"\n{'='*80}\nDocument {i}:\n{'='*80}\n")
    print(doc.text_resource.text)


Document 1:

# Multipurpose EnergyMax USB-RS Sensors

High Damage Threshold, Large Area, Laser Energy Sensors for a Wide Variety of Uses

The Multipurpose EnergyMax measure laser pulse energy over a wide range of wavelengths (190 nm to 12 µm), beam diameters, average power levels and repetition rates. They incorporate Coherent's unique MaxBlack coating, which offers high damage resistance and mechanical durability characteristics.

These sensors have USB and RS-232 interfaces for use with a PC or industrial controller. DB25 models are also available for use with stand- alone energy meters.

FEATURES
* MaxBlack coating delivers high damage threshold and mechanical durability
* Operate over the 190 nm to 12 µm range
* Enable pulse energy measurements from 300 nj to 2 J
* Measure single shot to 1 kHz repetition rate
* USB and RS-232 interfaces

APPLICATIONS
* Medical
* Scientific
* Industrial

Document 2:

Multipurpose Energy Max USB-RS Sensors Datasheet

SPECIFICATIONS

|               

In [ ]:
import concurrent.futures
import time

def process_documents_parallel(file_list, parser_template, max_workers=8, max_retries=5, timeout_seconds=120):
    results = {}
    
    # Define the worker function that will process each document
    def process_single_doc(fname):
        # Create a fresh parser instance for each document with the timeout set
        parser = LlamaParse(
            **{k: v for k, v in parser_template.__dict__.items() 
               if not k.startswith('_') and k != 'custom_client'}
        )
        
        # Set the timeout specifically for this job
        parser.job_timeout_in_seconds = timeout_seconds
        
        for attempt in range(max_retries):
            try:
                print(f"Attempt {attempt+1} parsing {fname.name} (timeout: {timeout_seconds}s)...")
                start_time = time.time()
                
                # Create a fresh thread/event loop for each document
                def parse_single():
                    return parser.aload_data(fname)
                
                doc = run_in_thread(parse_single())
                
                elapsed = time.time() - start_time
                
                # If we got here and doc has content, parsing worked
                if doc and len(doc) > 0: 
                    print(f"Successfully parsed {fname.name} in {elapsed:.2f} seconds")
                    return doc
                else:
                    print(f"No content returned on attempt {attempt+1}")
            except Exception as e:
                print(f"Error on attempt {attempt+1} for {fname.name}: {str(e)}")
                time.sleep(1)  # Brief pause before retry
                
        print(f"Failed to parse {fname.name} after {max_retries} attempts")
        return None
    
    # Use ThreadPoolExecutor to run multiple documents in parallel
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all jobs and map them to their filenames
        future_to_file = {executor.submit(process_single_doc, fname): fname for fname in file_list}
        
        # Process results as they complete
        for future in concurrent.futures.as_completed(future_to_file):
            fname = future_to_file[future]
            try:
                doc_result = future.result()
                if doc_result:
                    results[fname] = doc_result
                    print(f"✅ {fname.name}: {len(doc_result)} sections")
                else:
                    print(f"❌ {fname.name}: Failed to parse")
            except Exception as e:
                print(f"❌ {fname.name}: Exception occurred: {str(e)}")
    
    return results

In [ ]:
# Usage
all_results = process_documents_parallel(post_fnames, gpt4o_parser, max_workers=8, timeout_seconds=120)

# Print summary
print(f"\nSuccessfully processed {len(all_results)} out of {len(post_fnames)} documents")
for fname, doc in all_results.items():
    print(f"- {fname.name}: {len(doc)} sections")

Attempt 1 parsing labmax-touch-ds.pdf (timeout: 120s)...
Attempt 1 parsing multipurpose-energymax-sensors-usb-rs232-ds.pdf (timeout: 120s)...
Attempt 1 parsing pm10k-plus-ds.pdf (timeout: 120s)...
Attempt 1 parsing Understanding-ISO-17025-for-Laser-Power-and-Energy-Measurement-043520ed-a855-43f6-b05c-a9c83afded79.pdf (timeout: 120s)...
Started parsing the file under job_id 35335807-c0b6-454c-885d-2712f4505318
Started parsing the file under job_id 0254d454-0dce-4858-9412-76e9428e282b
Started parsing the file under job_id 0dba1c3d-ddb2-418b-b9f7-59454bc4d4c9
Started parsing the file under job_id 335138b9-e433-47db-ba99-176e35a3ea26
Successfully parsed pm10k-plus-ds.pdf in 17.79 seconds
✅ pm10k-plus-ds.pdf: 4 sections
Successfully parsed labmax-touch-ds.pdf in 21.85 seconds
✅ labmax-touch-ds.pdf: 3 sections
Successfully parsed multipurpose-energymax-sensors-usb-rs232-ds.pdf in 21.87 seconds
✅ multipurpose-energymax-sensors-usb-rs232-ds.pdf: 5 sections
.Successfully parsed Understanding-IS

In [ ]:
def view_all_documents(results_dict):
    """
    Display all documents in a nicely formatted way.
    
    Args:
        results_dict: Dictionary with filenames as keys and document lists as values
    """
    doc_counter = 1
    
    # Sort by filename to ensure consistent order
    for fname, doc_list in sorted(results_dict.items()):
        print(f"\n{'#'*100}\nFile: {fname.name}\n{'#'*100}")
        
        # Loop through each document in this file
        for i, doc in enumerate(doc_list, 1):
            print(f"\n{'='*80}\nDocument {i} of {len(doc_list)}:\n{'='*80}\n")
            print(doc.text_resource.text)
            doc_counter += 1
            
        print(f"\n{'~'*80}\nEnd of file: {fname.name}\n{'~'*80}\n")
    
    print(f"\nTotal documents displayed: {doc_counter-1}")

# Use it on your results
view_all_documents(all_results)


####################################################################################################
File: Understanding-ISO-17025-for-Laser-Power-and-Energy-Measurement-043520ed-a855-43f6-b05c-a9c83afded79.pdf
####################################################################################################

Document 1 of 5:

# Understanding ISO 17025 for Laser Power and Energy Measurement

**Coherent is accredited to ISO/IEC 17025:2005, the industry recognized standard that defines management and technical requirements for a calibration laboratory.**

## Introduction

ISO/IEC 17025 accreditation assures customers that laser manufacturers are producing calibrations to the highest standards. Updated specifications and enhanced calibration certificates provide additional clarity to uncertainty specifications, helping customers better understand the accuracy and repeatability that products can deliver.

Most manufacturers of laser power and energy measurement instruments calibrate the

In [ ]:
# View just one file by name
specific_file = [f for f in all_results.keys() if "multipurpose" in f.name][0]
view_all_documents({specific_file: all_results[specific_file]})


####################################################################################################
File: multipurpose-energymax-sensors-usb-rs232-ds.pdf
####################################################################################################

Document 1 of 5:

# Multipurpose EnergyMax USB-RS Sensors

## High Damage Threshold, Large Area, Laser Energy Sensors for a Wide Variety of Uses

The Multipurpose EnergyMax measure laser pulse energy over a wide range of wavelengths (190 nm to 12 μm), beam diameters, average power levels and repetition rates. They incorporate Coherent’s unique MaxBlack coating, which offers high damage resistance and mechanical durability characteristics.

These sensors have USB and RS-232 interfaces for use with a PC or industrial controller. DB25 models are also available for use with stand-alone energy meters.

## Features

- MaxBlack coating delivers high damage threshold and mechanical durability
- Operate over the 190 nm to 12 μm range
- Enabl

In [ ]:
import pickle
from llama_index.core import Document

def convert_to_standard_docs(docs_dict):
    standard_docs = []
    for file_path, doc_list in docs_dict.items():
        file_name = file_path.name  # Get just the filename
        total_docs = len(doc_list)
        
        for i, doc in enumerate(doc_list, 1):
            # Create standard Document object with enhanced metadata
            standard_doc = Document(
                text=doc.text_resource.text,
                metadata={
                    "source": str(file_path),
                    "file_name": file_name,
                    "doc_num": i,
                    "total_docs": total_docs,
                    "id": doc.id_ if hasattr(doc, 'id_') else None
                }
            )
            standard_docs.append(standard_doc)
    
    return standard_docs

# Save as standard documents
standard_docs = convert_to_standard_docs(all_results)
with open("static/parsed_lmc_docs.pkl", 'wb') as f:
    pickle.dump(standard_docs, f)

In [ ]:
import pickle
from pathlib import Path

def save_docs_to_pickle(docs_dict, file_path="static/parsed_lmc_docs.pkl"):
    """
    Save parsed documents dictionary to a pickle file.
    
    Args:
        docs_dict: Dictionary with filenames as keys and document lists as values
        file_path: Path to save the pickle file
    """
    # Make sure parent directory exists
    Path(file_path).parent.mkdir(parents=True, exist_ok=True)
    
    # Save to pickle file
    with open(file_path, 'wb') as f:
        pickle.dump(docs_dict, f)
    
    print(f"Saved {sum(len(docs) for docs in docs_dict.values())} documents from {len(docs_dict)} files to {file_path}")

In [ ]:
import pickle

def load_docs_from_pickle(file_path="static/parsed_lmc_docs.pkl"):
    """
    Load documents from a pickle file.
    
    Args:
        file_path: Path to the pickle file
        
    Returns:
        List of Document objects
    """
    with open(file_path, 'rb') as f:
        loaded_docs = pickle.load(f)
    
    print(f"Loaded {len(loaded_docs)} documents from {file_path}")
    return loaded_docs

# Usage
loaded_docs = load_docs_from_pickle()

Loaded 17 documents from static/parsed_lmc_docs.pkl


In [ ]:
def view_loaded_documents(docs, max_docs=None, max_chars=1000):
    """
    Display loaded documents in a readable format.
    
    Args:
        docs: List of Document objects
        max_docs: Maximum number of documents to display (None for all)
        max_chars: Maximum characters to display from each document
    """
    docs_to_display = docs[:max_docs] if max_docs else docs
    
    for i, doc in enumerate(docs_to_display, 1):
        print(f"\n{'='*80}\nDocument {i} of {len(docs_to_display)}:\n{'='*80}")
        
        # Print metadata
        print("\nMETADATA:")
        for key, value in doc.metadata.items():
            print(f"  {key}: {value}")
        
        # Print truncated text content
        print(f"\nCONTENT (truncated to {max_chars} chars):")
        text = doc.text[:max_chars]
        if len(doc.text) > max_chars:
            text += "..."
        print(text)
        
    print(f"\nDisplayed {len(docs_to_display)} of {len(docs)} documents")

# View the first 3 documents with truncated content
view_loaded_documents(loaded_docs, max_docs=3, max_chars=500)


Document 1 of 3:

METADATA:
  source: static/pm10k-plus-ds.pdf
  file_name: pm10k-plus-ds.pdf
  doc_num: 1
  total_docs: 4
  id: 46bcba32-1732-4fd7-853b-dec6c7d71d3d

CONTENT (truncated to 500 chars):
# PM10K+ LASER POWER SENSOR

## Large Area Water-Cooled kW Sensors

These 10 kW water-cooled laser power sensors feature a 65 mm x 65 mm sensor with the BB+ coating that can handle power densities from 6 kW/cm² (at 1 kW) to 2.7 kW/cm² (at 10 kW). The sensor can measure up to 10 kW continuously and up to 12 kW on an intermittent basis. You can choose from USB + DB-25 or RS-232 communication options. DB-25 + USB sensor models are compatible with Coherent’s stand-alone power meters, which can be or...

Document 2 of 3:

METADATA:
  source: static/pm10k-plus-ds.pdf
  file_name: pm10k-plus-ds.pdf
  doc_num: 2
  total_docs: 4
  id: 09422a57-a8fb-42d5-b8f1-95ea79b2a9c7

CONTENT (truncated to 500 chars):
# PM10K+ LASER POWER SENSOR

| Specifications                        | PM10K+               

In [ ]:
loaded_docs = load_docs_from_pickle()

Loaded 17 documents from 4 files


In [ ]:
view_all_documents(loaded_docs)


####################################################################################################
File: Understanding-ISO-17025-for-Laser-Power-and-Energy-Measurement-043520ed-a855-43f6-b05c-a9c83afded79.pdf
####################################################################################################

Document 1 of 5:

# Understanding ISO 17025 for Laser Power and Energy Measurement

**Coherent is accredited to ISO/IEC 17025:2005, the industry recognized standard that defines management and technical requirements for a calibration laboratory.**

## Introduction

ISO/IEC 17025 accreditation assures customers that laser manufacturers are producing calibrations to the highest standards. Updated specifications and enhanced calibration certificates provide additional clarity to uncertainty specifications, helping customers better understand the accuracy and repeatability that products can deliver.

Most manufacturers of laser power and energy measurement instruments calibrate the

In [ ]:
!ls static

Understanding-ISO-17025-for-Laser-Power-and-Energy-Measurement-043520ed-a855-43f6-b05c-a9c83afded79.pdf
labmax-touch-ds.pdf
multipurpose-energymax-sensors-usb-rs232-ds.pdf
parsed_lmc_docs.pkl
pm10k-plus-ds.pdf
